In [161]:
import joblib
import pandas as pd

# 🚀 Carregar o modelo treinado e as colunas do X_train
modelo_rf = joblib.load("modelo_rf_otimizado.pkl")  # Ajuste para o modelo correto se necessário
X_train_columns = joblib.load("X_train_columns.pkl")

# Criar um DataFrame vazio com as colunas corretas
X_train = pd.DataFrame(columns=X_train_columns)

print(f"✅ Modelo e dados carregados. Total de colunas em X_train: {len(X_train_columns)}")

import joblib
import pandas as pd

# 🚀 Carregar df_videos salvo do treinamento
df_videos = joblib.load("df_videos.pkl")

# 🚀 Calcular automaticamente a média de views dos últimos 10 vídeos
media_views_ultimos_10 = df_videos["views"].rolling(window=10, min_periods=1).mean().iloc[-1]

print(f"📊 Média de views dos últimos 10 vídeos: {int(media_views_ultimos_10)}")

✅ Modelo e dados carregados. Total de colunas em X_train: 110
📊 Média de views dos últimos 10 vídeos: 39720


In [162]:
# 🚀 Lista de palavras-chave do nicho (ajuste conforme necessário)
palavras_chave_nicho = ["treino", "hipertrofia", "massa muscular", "exercício", "dieta", "proteína", "suplemento"]

# 🚀 Criar uma função para contar palavras do nicho no título do novo vídeo
def contar_palavras_nicho(titulo):
    return sum(1 for palavra in palavras_chave_nicho if palavra in titulo.lower())

In [163]:
import cv2
import numpy as np
from PIL import Image

# 🚀 Função para analisar a thumbnail localmente
def analisar_thumbnail_local(caminho_imagem):
    try:
        # Carregar imagem
        img = Image.open(caminho_imagem)
        img_cv = np.array(img)

        # Converter para escala de cinza
        gray = cv2.cvtColor(img_cv, cv2.COLOR_RGB2GRAY)

        # Calcular brilho (média dos valores dos pixels)
        brilho = np.mean(gray)

        # Calcular contraste
        contraste = gray.std()

        # Contar faces na imagem (usando um modelo pré-treinado do OpenCV)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        qtd_faces = len(faces)

        # Extrair cor predominante
        pixels = img_cv.reshape(-1, 3)
        cor_predominante = np.mean(pixels, axis=0).mean()

        return brilho, contraste, cor_predominante, qtd_faces

    except Exception as e:
        print(f"Erro ao processar a imagem: {e}")
        return None, None, None, None


In [164]:

# 🚀 Caminho da imagem no seu PC (exemplo)
caminho_thumb = "/Users/luisfelipesouzacardoso/Documents/Estudo/Data_Science/fdm_analytics/250129_defesa-pessoal-mulheresjpg.jpg"  # Substitua pelo caminho real

# 🚀 Analisar a thumbnail carregada do PC
brilho, contraste, cor_predominante, qtd_faces = analisar_thumbnail_local(caminho_thumb)

# 🚀 Atualizar `novo_video` com as features extraídas automaticamente
novo_video["brilho"] = brilho
novo_video["contraste"] = contraste
novo_video["cor_predominante"] = cor_predominante
novo_video["qtd_faces"] = qtd_faces

# Exibir o DataFrame atualizado
print(novo_video)

   likes  comments  duration_sec  year  month  day  weekday  likes_per_view  \
0      0         0          1700  2025      2    4        0               0   

   comments_per_view  title_word_count  ...  tortuguita  treinar  um  usar  \
0                  0                 9  ...           0        0   0     0   

   você  vs    brilho  contraste  cor_predominante  qtd_faces  
0     0   0  84.65458  77.445994         84.455341          3  

[1 rows x 114 columns]


In [165]:
# Definir o título do novo vídeo
titulo_novo_video = "Leo Stronda e Scarpelly | Fala Monstro Talk Show #01"

# 🚀 Calcular automaticamente a média de views dos últimos 10 vídeos
media_views_ultimos_10 = df_videos["views"].rolling(window=10, min_periods=1).mean().iloc[-1]


# 🚀 Criar o DataFrame do novo vídeo com todas as features
novo_video = pd.DataFrame({
    "title": [titulo_novo_video],  
    "duration_sec": [1700],  
    "year": [2025],
    "month": [2],
    "day": [4],
    "weekday": ["Tuesday"],  
    "likes": [0],  
    "comments": [0],
    "likes_per_view": [0],
    "comments_per_view": [0],
    "title_word_count": [len(titulo_novo_video.split())],  
    "qtd_palavras_thumb": [3],  
    "media_views_ultimos_10": [media_views_ultimos_10],  
    "media_likes_per_view": [0.1],
    "media_comments_per_view": [0.02],
    "qtd_palavras_nicho": [palavras_chave_nicho ]  
})

# 🚀 Adicionar os valores da thumbnail analisada
novo_video["brilho"] = brilho
novo_video["contraste"] = contraste
novo_video["cor_predominante"] = cor_predominante
novo_video["qtd_faces"] = qtd_faces

# Exibir o DataFrame atualizado para verificar
print(novo_video)

                                               title  duration_sec  year  \
0  Leo Stronda e Scarpelly | Fala Monstro Talk Sh...          1700  2025   

   month  day  weekday  likes  comments  likes_per_view  comments_per_view  \
0      2    4  Tuesday      0         0               0                  0   

   title_word_count  qtd_palavras_thumb  media_views_ultimos_10  \
0                10                   3                 39720.9   

   media_likes_per_view  media_comments_per_view  \
0                   0.1                     0.02   

                                  qtd_palavras_nicho    brilho  contraste  \
0  [treino, hipertrofia, massa muscular, exercíci...  84.65458  77.445994   

   cor_predominante  qtd_faces  
0         84.455341          3  


In [166]:

# 🚀 Verificar se a coluna "weekday" existe no novo_video
if "weekday" in novo_video.columns:
    novo_video = pd.get_dummies(novo_video, columns=["weekday"])
else:
    print("⚠️ A coluna 'weekday' não está presente. Criando colunas manualmente.")

    # Criar colunas one-hot encoding manualmente (todas zeradas por padrão)
    for dia in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:
        novo_video[f"weekday_{dia}"] = 0

    # Definir o dia correto como 1
    novo_video[f"weekday_Monday"] = 1  # Ajuste conforme o dia real do vídeo

# 🚀 Garantir que todas as colunas do treinamento estão presentes
for col in X_train.columns:
    if col not in novo_video.columns:
        novo_video[col] = 0  # Adicionar colunas faltantes com valor 0

# 🚀 Reordenar as colunas para corresponder ao treinamento
novo_video = novo_video[X_train.columns]

# 🚀 Converter para valores numéricos
novo_video = novo_video.apply(pd.to_numeric, errors="coerce").fillna(0)

print(f"✅ Novo vídeo formatado para previsão: {novo_video.shape[1]} features")


# 🚀 Converter o dia da semana para one-hot encoding
novo_video = pd.get_dummies(novo_video, columns=["weekday"])

# Garantir que todas as colunas do treinamento estão presentes
for col in X_train.columns:
    if col not in novo_video.columns:
        novo_video[col] = 0  # Adicionar colunas faltantes com valor 0

# Reordenar as colunas para corresponder ao treinamento
novo_video = novo_video[X_train.columns]

# Converter para valores numéricos
novo_video = novo_video.apply(pd.to_numeric, errors="coerce").fillna(0)

print(f"✅ Novo vídeo formatado para previsão: {novo_video.shape[1]} features")

✅ Novo vídeo formatado para previsão: 110 features
✅ Novo vídeo formatado para previsão: 110 features


/var/folders/d6/gf2wlh6x083f9l84m7czsqbw0000gn/T/ipykernel_84282/4090742985.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  novo_video[col] = 0  # Adicionar colunas faltantes com valor 0
/var/folders/d6/gf2wlh6x083f9l84m7czsqbw0000gn/T/ipykernel_84282/4090742985.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  novo_video[col] = 0  # Adicionar colunas faltantes com valor 0
/var/folders/d6/gf2wlh6x083f9l84m7czsqbw0000gn/T/ipykernel_84282/4090742985.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [167]:
# 🚀 Converter o dia da semana para one-hot encoding
novo_video = pd.get_dummies(novo_video, columns=["weekday"])

# Garantir que todas as colunas do treinamento estão presentes
for col in X_train.columns:
    if col not in novo_video.columns:
        novo_video[col] = 0  # Adicionar colunas faltantes com valor 0

# Reordenar as colunas para corresponder ao treinamento
novo_video = novo_video[X_train.columns]

# Converter tudo para valores numéricos
novo_video = novo_video.apply(pd.to_numeric, errors="coerce").fillna(0)

# ✅ Exibir a estrutura do novo vídeo
print(f"✅ Novo vídeo formatado para previsão: {novo_video.shape[1]} features")


✅ Novo vídeo formatado para previsão: 110 features


In [168]:
# 🚀 Converter para DataFrame e garantir que as colunas estão corretas
novo_video_df = pd.DataFrame(novo_video, columns=X_train.columns)

# 🚀 Converter para NumPy apenas na previsão para evitar warnings
novo_video_array = novo_video_df.to_numpy()

# 🚀 Fazer a previsão corretamente
previsao_visualizacoes = modelo_rf.predict(novo_video_array)  # Agora passa um array puro

# 📊 Exibir a previsão
print(f"📊 Previsão de Visualizações: {int(previsao_visualizacoes[0])}")

📊 Previsão de Visualizações: 12193
